# Readout weight calibration

In this notebook, you will learn how to calibrate and use optimal integration weights to distinguish between qubits states in circuit QED.

This demonstration runs without connection to real qubits, assuming a loopback on the readout drive line directly into the readoud acquisition line. We emulate the measurement signals corresponding to different qubit states by two different measurement pulses, differing only by a phase. ties of the instrument.  

## 0. General Imports and Definitions

### 0.1 Python Imports 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# all LabOne Q functionality
from laboneq.simple import *

# Helpers:
from laboneq.contrib.example_helpers.feedback_helper import (
    complex_freq_phase,
    exp_raw,
    exp_integration,
    exp_discrimination,
)

from laboneq.contrib.example_helpers.generate_example_datastore import (
    generate_example_datastore,
    get_first_named_entry,
)

In [ ]:
# Build an in-memory data store with device setup and qubit parameters for the
# example notebooks
dummy_db = generate_example_datastore(in_memory=True)

use_emulation = True


# 1. Device setup and calibration

## 1.1 Load a calibrated Device Setup and qubit object

In [ ]:
my_setup = get_first_named_entry(
    db=dummy_db, name="12_qubit_setup_shfsg_shfqa_shfqc_hdawg_pqsc_calibrated"
)

my_qubit = get_first_named_entry(db=dummy_db, name="fixed_transmon_0")

q0 = my_setup.logical_signal_groups["q0"].logical_signals
q1 = my_setup.logical_signal_groups["q1"].logical_signals

## 1.2 Adapt setup calibration

In this notebook we are using a pulse played from a second measure line to emulate the qubit being in the excited state. In this case we want to have the same instrument settings for the two used measurement lines. 
Additionally, for the method of readout weight calibration demonstrated in this notebook, the acquire line should not be modulated, as the calculated readout weights already contain the software modulation by construction.

In [ ]:
readout_weight_calibration = Calibration()
readout_weight_calibration[
    "/logical_signal_groups/q1/measure_line"
] = my_setup.get_calibration()["/logical_signal_groups/q0/measure_line"]
readout_weight_calibration[
    "/logical_signal_groups/q0/acquire_line"
] = my_setup.get_calibration()["/logical_signal_groups/q0/acquire_line"]
readout_weight_calibration["/logical_signal_groups/q0/acquire_line"].oscillator = None

# print(readout_weight_calibration)

my_setup.set_calibration(readout_weight_calibration)

# print(my_setup.get_calibration())


In [ ]:
# create and connect to a LabOne Q session
my_session = Session(device_setup=my_setup)
my_session.connect(do_emulation=use_emulation)


## 2. Calibration of readout weigths for state discrimination - two qubit states, manual calculation of readout weights

We determine the optimal integration weights by subtracting and conjugating the raw response corresponding to the two different qubit states. We then additionally rotate these integration weights to result in maximum separation of the resulting IQ values on the real axis and adjust the corresponding threshold value in the device setup calibration.

### 2.1 Define measurement pulse waveforms to simulate the return from a qubits |0> and |1> states

In [ ]:
# measure pulse parameters
pulse_len = my_qubit.parameters.user_defined["readout_length"]
pulse_phase = np.pi / 4

# sampling rate of SHF instruments
sampling_rate = 2.0e9

pulse_freq = 0.0
measure0_gen2 = pulse_library.sampled_pulse_complex(
    complex_freq_phase(
        sampling_rate,
        pulse_len,
        pulse_freq,
        my_qubit.parameters.user_defined["readout_amplitude"],
        0,
    )
)
measure1_gen2 = pulse_library.sampled_pulse_complex(
    complex_freq_phase(
        sampling_rate,
        pulse_len,
        pulse_freq,
        my_qubit.parameters.user_defined["readout_amplitude"],
        pulse_phase,
    )
)

### 2.2 Determine optimal integration weights based on raw readout results of two measurement pulses

In [ ]:
## Raw |0>
r = my_session.run(exp_raw(measure_pulse=measure0_gen2, q0=q0, pulse_len=pulse_len))
raw0 = r.acquired_results["raw"].data

## Raw |1>
r = my_session.run(exp_raw(measure_pulse=measure1_gen2, q0=q0, pulse_len=pulse_len))
raw1 = r.acquired_results["raw"].data

## optimal integration kernel
samples_kernel = np.conj(raw1 - raw0)
# plt.figure()
# plt.plot(samples_kernel.real, samples_kernel.imag)
plt.figure()
plt.plot(samples_kernel.real)
plt.plot(samples_kernel.imag)

### 2.3 Determine optimal rotation of integration weights and discrimination threshold

In [ ]:
do_rotation = True

my_exp = exp_integration(
    measure0=measure0_gen2,
    measure1=measure1_gen2,
    q0=q0,
    q1=q1,
    samples_kernel=samples_kernel,
)

r = my_session.run(my_exp)
res0 = r.acquired_results["data0"].data
res1 = r.acquired_results["data1"].data

connect_vector = np.median(res1) - np.median(res0)
if do_rotation:
    rotation_angle = -np.angle(connect_vector)
else:
    rotation_angle = 0

res0_rot = res0 * np.exp(1j * rotation_angle)
res1_rot = res1 * np.exp(1j * rotation_angle)

my_threshold = (np.median(res0_rot.real) + np.median(res1_rot.real)) / 2

if do_rotation:
    plt.scatter(res0.real, res0.imag, c="k", alpha=0.1)
    plt.scatter(res1.real, res1.imag, c="g", alpha=0.1)

plt.scatter(res0_rot.real, res0_rot.imag, c="b")
plt.scatter(res1_rot.real, res1_rot.imag, c="r")
plt.plot(
    [my_threshold, my_threshold],
    [
        min([*res0_rot.imag, *res1_rot.imag, *res0.imag, *res1.imag]),
        max([*res0_rot.imag, *res1_rot.imag, *res0.imag, *res1.imag]),
    ],
    "r",
)
if do_rotation:
    print(f"Using threshold = {my_threshold:e} and rotation angle: {rotation_angle:e}")
else:
    print(f"Using threshold={my_threshold:e}")

In [ ]:
## define properly rotated integration kernel and set state discrimination threshold in device setup calibration
my_integration_weights = pulse_library.sampled_pulse_complex(
    samples_kernel * np.exp(1j * rotation_angle)
)

q0["acquire_line"].calibration.threshold = my_threshold

### 2.4 Check status of state discrimination calibration

#### 2.4.1 Check for proper rotation of kernel

IQ values should be maximally separate on the real axis

In [ ]:
my_other_exp = exp_integration(
    measure0=measure0_gen2,
    measure1=measure1_gen2,
    q0=q0,
    q1=q1,
    samples_kernel=samples_kernel,
    rotation_angle=rotation_angle,
)

r = my_session.run(my_other_exp)

res0 = r.acquired_results["data0"].data
res1 = r.acquired_results["data1"].data

connect_vector = np.median(res1) - np.median(res0)

threshold_rot = (np.median(res0.real) + np.median(res1.real)) / 2

plt.scatter(res0.real, res0.imag, c="b")
plt.scatter(res1.real, res1.imag, c="r")

plt.plot(
    [threshold_rot, threshold_rot],
    [min([*res0.imag, *res1.imag]), max([*res0.imag, *res1.imag])],
    "r",
)

print(f"Using threshold={threshold_rot:e}")

#### 2.4.2 Check correct state discrimination when including rotation of integration weights

In [ ]:
r = my_session.run(
    exp_discrimination(
        measure0=measure0_gen2,
        measure1=measure1_gen2,
        q0=q0,
        q1=q1,
        samples_kernel=samples_kernel,
        threshold=my_threshold,
        rotation_angle=rotation_angle,
    )
)
s0 = r.acquired_results["data0"].data
s1 = r.acquired_results["data1"].data

plt.plot(s0.real, ".b")
plt.plot(s1.real, ".r")